# Tableau and the File System

The Langchain community provides out of the box tools that can make the Superstore Agent more powerful. Now that this agent is able to query Tableau data sources, we can give it tools to interact with the file system so it accomplish the following:

1. Write data outputs to the file system
2. Write insights to the file system
3. List outputs
4. Read outputs back to you
   
This is possible thanks to the [filesystem toolkit](https://python.langchain.com/docs/integrations/tools/filesystem/).

![Tableau dashboards inside bubbles](./assets/embed_samples.png)

Start by importing the necessary packages:

In [28]:
import os

from langchain_community.agent_toolkits import FileManagementToolkit

# Initialize the File System Toolkit

The next step is to create a temporary working directory and initialize the File System toolkit. That way we can get then tools we need for this exercise such as `read_tool`, `write_tool` and `list_tool`.

When combined with data from Tableau the Superstore Agent will be capable of several interesting scenarios.

In [36]:
# Use "temp" folder in the current Jupyter Notebook directory
temp_dir = "temp"

# Create the temp directory if it doesn't exist
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

#  initialize the desired toolkit
toolkit = FileManagementToolkit(
    root_dir=temp_dir,
    selected_tools=["read_file", "write_file", "list_directory"]
)

# store all of the tools in the toolkit as 'tools'
tools = toolkit.get_tools()

# unpack the tools we will use in the exercise
read_tool, write_tool, list_tool, *_ = tools

# Testing the Filesystem Toolkit

To get acquainted with these new tools, let's "invoke" them individually without an agent to inspect their output.

Let's first start with writing a new file, you can inspect the physical output yourself inside of the `experimental/notebooks/temp` folder:

In [41]:
# Create the file before writing to it
file_path = os.path.join(temp_dir, "example.txt")
with open(file_path, "w") as f:
    pass  # Create an empty file

write_tool.invoke({"file_path": "hello_superstore.txt", "text": "Hello Superstore!"})

'File written successfully to hello_superstore.txt.'

Now list the files inside of the `experimental/notebooks/temp` folder with the `list_tool`:

In [43]:
# List files in the working directory using `list_tool`
files = list_tool.invoke({})

# Split the raw output into a list of filenames, removing empty entries
file_list = [name for name in files.strip().split('\n') if name]

# Format the list into Markdown list items
markdown_list_items = []
if file_list:
    for file_name in sorted(file_list): # Sort alphabetically for consistency
        markdown_list_items.append(f"* {file_name}") # Prepend Markdown list marker '* '
        # You could use "- " instead: markdown_list_items.append(f"- {file_name}")
else:
    markdown_list_items.append("* (Directory is empty)") # Indicate if empty

# Join the Markdown list items back into a single string
markdown_output = "\n".join(markdown_list_items)

# Print the final Markdown formatted string
print("\nFormatted Markdown List Output:")
print(markdown_output)


Formatted Markdown List Output:
* .gitignore
* .gitkeep
* hello_superstore.txt


NOTE: `.gitkeep` & `.gitignore` were already present inside the "temp" folder to make it a temporary placeholder

Lets inspect the contents of the `hello_superstore.txt` file with the `read_tool`:

In [44]:
# Invoke the read_tool, providing the relative path within the root_dir
file_content = read_tool.invoke({"file_path": "hello_superstore.txt"})

# The result is the content of the file as a string
print("\n--- Successfully Read File Content ---")
print(file_content)
print("--- End of File Content ---")


--- Successfully Read File Content ---
Hello Superstore!
--- End of File Content ---
